# KPI Scenarios (SMED, TPM, FPY)
What-if analysis on the synthetic dataset to show potential gains from:
- **SMED**: reduce changeover by 15 minutes
- **TPM**: reduce downtime by 10%
- **FPY**: improve quality by +2 percentage points


In [ ]:
import pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
CSV = Path(__file__).resolve().parents[1] / 'data-samples' / 'line_production.csv'
df = pd.read_csv(CSV, parse_dates=['date'])
ideal_rate = {"BodyShop": 0.5, "Assembly": 0.8}

def compute_oee(d):
    d = d.copy()
    d['planned_effective'] = d['planned_time_min'] - d['changeover_min']
    d['availability'] = (d['run_time_min'] / d['planned_effective']).clip(upper=1)
    d['performance'] = ((d['good_units'] + d['scrap_units']) / (d['line'].map(ideal_rate) * d['run_time_min'])).clip(upper=1)
    d['quality'] = (d['good_units'] / (d['good_units'] + d['scrap_units'])).fillna(0)
    d['oee'] = d['availability'] * d['performance'] * d['quality']
    return d

base = compute_oee(df)

# Scenarios
smed = df.copy(); smed['changeover_min'] = (smed['changeover_min'] - 15).clip(lower=0)
smed = compute_oee(smed)

tpm = df.copy(); tpm['downtime_min'] = (tpm['downtime_min'] * 0.9).round()
tpm['run_time_min'] = tpm['planned_time_min'] - tpm['changeover_min'] - tpm['downtime_min']
tpm = compute_oee(tpm)

fpy = df.copy();
fpy['good_units'] = (fpy['good_units'] * 1.02).round().astype(int)
fpy = compute_oee(fpy)

summary = pd.DataFrame({
    'baseline_oee': base['oee'].mean(),
    'smed_oee': smed['oee'].mean(),
    'tpm_oee': tpm['oee'].mean(),
    'fpy_oee': fpy['oee'].mean()
}, index=['mean']).T
summary

In [ ]:
import matplotlib.pyplot as plt
labels = ['Baseline','SMED','TPM','FPY']
values = [
    summary.loc['baseline_oee','mean'],
    summary.loc['smed_oee','mean'],
    summary.loc['tpm_oee','mean'],
    summary.loc['fpy_oee','mean']
]
plt.figure()
plt.bar(labels, values)
plt.ylabel('OEE (0–1)')
plt.title('OEE – What-if Scenarios')
plt.show()